In [1]:
from decouple import config

In [2]:
assert config("POLYGON_API_KEY", default=None, cast=str) is not None

In [3]:
POLYGON_API_KEY = config("POLYGON_API_KEY", default=None, cast=str)

In [4]:
import requests

# check polygon.io api paths formats

# polygon api path format = /v2/aggs/ticker/{stocksTicker}/range/{multiplier}/{timespan}/{from}/{to}
ticker = "AAPL"  # company's symbol in stocks
multiplier = "1"
timespan = "day"
from_date = "2024-11-02"
to_date = "2024-12-29"
path = f"/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}"
url = f"https://api.polygon.io{path}?apiKey={POLYGON_API_KEY}"

response = requests.get(url)
data = response.json()

In [5]:
result = data['results'][0]
result

{'v': 40921729.0,
 'vw': 221.7547,
 'o': 220.99,
 'c': 222.01,
 'h': 222.79,
 'l': 219.71,
 't': 1730696400000,
 'n': 647102}

In [6]:
from datetime import datetime
import pytz

def transform_polygon_result(result):
    unix_timestamp = result.get('t')/1000
    utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone("UTC"))
    #print(utc_timestamp)
    return{
        "open_price" : result["o"],
        "close_price" : result["c"],
        "high_price" : result["h"],
        "low_price" : result["l"],
        "number_of_trades" : result["n"],
        "volume" : result["v"],
        "volume_weighted_average" : result["vw"],
        "time" : utc_timestamp,
    }

transform_polygon_result(result)

{'open_price': 220.99,
 'close_price': 222.01,
 'high_price': 222.79,
 'low_price': 219.71,
 'number_of_trades': 647102,
 'volume': 40921729.0,
 'volume_weighted_average': 221.7547,
 'time': datetime.datetime(2024, 11, 4, 5, 0, tzinfo=<UTC>)}

<h2>Create a Custom Polygon API Client</h2>

In [11]:
from dataclasses import dataclass
from typing import Literal
from urllib.parse import urlencode

POLYGON_API_KEY = config("POLYGON_API_KEY", default=None, cast=str)

@dataclass
class PolygonAPIClient:
    ticker: str = "AAPL"  # company's symbol in stocks
    multiplier: int = 1
    timespan: str = "day"
    from_date: str = "2024-11-20"
    to_date: str = "2024-12-03"
    api_key: str = ""
    adjusted: bool = True
    sort: Literal["asc", "desc"] = "asc"

    def get_api_key(self):
        return self.api_key or POLYGON_API_KEY
    
    def get_headers(self):
        api_key = self.get_api_key()
        return {
            "Authorization" : f"Bearer {api_key}"
        }

    def get_params(self):
        return {
            "adjusted" : self.adjusted,
            "sort" : self.sort
        }

    def generate_url(self, pass_auth=False):
        path = f"/v2/aggs/ticker/{self.ticker}/range/{self.multiplier}/{self.timespan}/{self.from_date}/{self.to_date}"
        url = f"https://api.polygon.io{path}"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        if pass_auth:
            api_key = self.get_api_key()
            url += f"&apiKey={api_key}"
        return url

    def fetch_data(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status() # not 200/201. Incase there is an error
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()
        results = data["results"]
        dataset = []
        for result in results:
            dataset.append(transform_polygon_result(result))
        return dataset

In [12]:
stock_api_client = PolygonAPIClient(ticker="GOOG", multiplier=1)

In [13]:
# stock_api_client.generate_url(pass_auth=True) # appends api key on the link
stock_api_client.generate_url()  # gets api key from .env file

'https://api.polygon.io/v2/aggs/ticker/GOOG/range/1/day/2024-11-20/2024-12-03?adjusted=True&sort=asc'

In [14]:
dataset = stock_api_client.get_stock_data()
dataset

[{'open_price': 178.83,
  'close_price': 177.33,
  'high_price': 179.11,
  'low_price': 175.33,
  'number_of_trades': 191057,
  'volume': 14429930.0,
  'volume_weighted_average': 176.8894,
  'time': datetime.datetime(2024, 11, 20, 5, 0, tzinfo=<UTC>)},
 {'open_price': 175.455,
  'close_price': 169.24,
  'high_price': 175.58,
  'low_price': 165.31,
  'number_of_trades': 531658,
  'volume': 36226068.0,
  'volume_weighted_average': 168.2341,
  'time': datetime.datetime(2024, 11, 21, 5, 0, tzinfo=<UTC>)},
 {'open_price': 167.16,
  'close_price': 166.57,
  'high_price': 168.2645,
  'low_price': 165.71,
  'number_of_trades': 321715,
  'volume': 23446733.0,
  'volume_weighted_average': 166.8668,
  'time': datetime.datetime(2024, 11, 22, 5, 0, tzinfo=<UTC>)},
 {'open_price': 167.99,
  'close_price': 169.43,
  'high_price': 170.46,
  'low_price': 167.4,
  'number_of_trades': 244289,
  'volume': 18989585.0,
  'volume_weighted_average': 169.2641,
  'time': datetime.datetime(2024, 11, 25, 5, 0, tz